기존 라이브러리

In [1]:
import torch
from torch import nn

model = nn.Sequential()
model.append(module = nn.Linear(in_features=784,
                                out_features=256,
                                bias =True))
model.append(module = nn.ReLU())
model.append(module = nn.Linear(in_features=256,
                                out_features=10,
                                bias =True))

Sequential(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)

subclassing을 활용한 모델

In [2]:
import torch
from torch import nn

In [3]:
class CustomModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear1 = nn.Linear(in_features=784,
                                out_features=256,
                                bias =True)
    self.linear2 = nn.Linear(in_features=256,
                                out_features=10,
                                bias =True)

    def forward(self, x):
      x = torch.relu(self.linear1(x))
      return self.linear2(x)

In [5]:
class CombModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear_stack = nn.Sequential(
        nn.Linear(in_features=784,
                                out_features=256,
                                bias =True),
        nn.ReLU(),
        nn.Linear(in_features=256,
                                out_features=10,
                                bias =True)
    )
    # 기타 다른 레이어 추가가능

  def forward(self):
    pass

#Pytorch Lighting

* 기존의 학습/검증 루프를 보다 더 간단하게 만들어주고, 모델 구성 및 실험에만 집중하게만 해줌


In [7]:
!pip install lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [10]:
import lightning.pytorch as pl

In [12]:
# 기존
class CustomModel(nn.Module):
  pass

# lightinging
class LCustomModel(pl.LightningModule):
  pass

#Subclassing과 Pytorch Lightning을 활용한 MNIST 실습

In [13]:
import torch
from torch import nn
# pytorch lightning
import lightning.pytorch as pl

# Data Management
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import MNIST
from torchvision import datasets, transforms

#모델 정의

In [16]:
class LMINSTModel(pl.LightningModule):
  def __init__(self):
    super().__init__()

    # MNIST 이미지의 크기 : (1, 28, 28) => (channels, width, height)
    self.linear1 = nn.Linear(in_features=784,
                                out_features=256,
                                bias =True)
    self.linear2 = nn.Linear(in_features=256,
                                out_features=10,
                                bias =True)


  def forward(self,x):
    batch_size, channels, width, height = x.size()

    # Flatten
    x = x.view(batch_size, -1) # (batch size, 1*28*28)

    # Layer1
    x = torch.relu(self.linear1(x))

    # Layer2
    x = self.linear2(x)

    x = torch.log_softmax(x, dim=1)
    return x

  def cross_entropy_loss(self, logits, labels):
    # softmax + nll_loss
    return nn.functional.nll_loss(input=logits, target=labels)

  def training_step(self, train_batch, batch_idx):
    x, y = train_batch
    logits = self.forward(x)
    loss = self.cross_entropy_loss(logits, y)
    self.log('train_loss', loss)
    return loss

  def validation_step(self, val_batch, batch_idx):
    x, y = val_batch
    logits = self.forward(x)
    loss = self.cross_entropy_loss(logits, y)
    self.log('val_loss', loss)

  def test_step(self, test_batch, batch_idx):
    x, y = test_batch
    logits = self.forward(x)
    loss = self.cross_entropy_loss(logits, y)
    self.log('test_loss', loss)

  def configure_optimizers(self):
    optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
    return optimizer

In [17]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.1307,), (0.3081,))])

# 데이터 준비
mnist_train=MNIST("./",
                  train=True,
                  download=True,
                  transform=transform)

mnist_train,  mnist_val = random_split(mnist_train, [0.9,0.1])
mnist_test,=MNIST("./",
                  train=False,
                  download=True,
                  transform=transform)

ValueError: too many values to unpack (expected 1)

In [ ]:
train_loader = DataLoader(mnist_train, batch_size=64, shuffle=True)
val_loader = DataLoader(mnist_val, batch_size=64)
test_loader = DataLoader(mnist_test, batch_size=64)

In [ ]:
model = LMINSTModel()
trainer = pl.Trainer(max_epochs=5)

trainer.fit(model, train_loader, val_loader)

In [ ]:
trainer.test(model, dataloaders=test_loader)